In [1]:
import pandas as pd
import numpy
from sklearn import model_selection, preprocessing

In [2]:
df = pd.read_csv('Unemployment_mental_illness_survey.csv')
df.head()

,I am currently employed at least part-time,I identify as having a mental illness,Education,I have my own computer separate from a smart phone,I have been hospitalized before for my mental illness,How many days were you hospitalized for your mental illness,I am legally disabled,I have my regular access to the internet,I live with my parents,I have a gap in my resume,...,Obsessive thinking,Mood swings,Panic attacks,Compulsive behavior,Tiredness,Age,Gender,Household Income,Region,Device Type
0,0,0,High School or GED,0,0,0.0,0,1,0,1,...,1.0,0.0,1.0,0.0,0.0,30-44,Male,"$25,000-$49,999",Mountain,Android Phone / Tablet
1,1,1,Some Phd,1,0,0.0,0,1,0,0,...,0.0,0.0,1.0,0.0,1.0,18-29,Male,"$50,000-$74,999",East South Central,MacOS Desktop / Laptop
2,1,0,Completed Undergraduate,1,0,0.0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,30-44,Male,"$150,000-$174,999",Pacific,MacOS Desktop / Laptop
3,0,0,Some Undergraduate,1,0,NaN,0,1,1,1,...,0.0,0.0,0.0,0.0,0.0,30-44,Male,"$25,000-$49,999",New England,Windows Desktop / Laptop
4,1,1,Completed Undergraduate,1,1,35.0,1,1,0,1,...,1.0,1.0,1.0,1.0,1.0,30-44,Male,"$25,000-$49,999",East North Central,iOS Phone / Tablet


In [11]:
clean_df=df.rename(columns={"I am currently employed at least part-time":"employed",
                   "I identify as having a mental illness":"mental_illness",
                   "I have my own computer separate from a smart phone":"own_computer",
                   "I have been hospitalized before for my mental illness":"hospitalized",
                   "How many days were you hospitalized for your mental illness":"hospitalized_days",
                   "I am legally disabled":"disabled","I have my regular access to the internet":"internet",
                   "I live with my parents":"with_parents","I have a gap in my resume":"gap",
                   "Total length of any gaps in my resume in\xa0months.":"months_gap",
                   "Annual income (including any social welfare programs) in USD":"income_with_social",
                   "I am unemployed":"unemployed",
                   "I read outside of work and school":"read",
                   "Annual income from social welfare programs":"welfare_income",
                   "I receive food stamps":"food_stamp",
                   "I am on section 8 housing":"section_8",
                   "How many times were you hospitalized for your mental illness":"times_hospitalized",
                   "Lack of concentration":"Lack_concentration",
                   "Obsessive thinking":"Obsessive_thinking",
                   "Mood swings":"Mood_swings",
                   "Panic attacks":"Panic_attacks",
                   "Compulsive behavior":"Compulsive_behavior",
                   "Household Income":"Household_Income",
                   "Device Type":"Device"
                  })
clean_df.head()
list(clean_df.columns)

['employed',
 'mental_illness',
 'Education',
 'own_computer',
 'hospitalized',
 'hospitalized_days',
 'disabled',
 'internet',
 'with_parents',
 'gap',
 'months_gap',
 'income_with_social',
 'unemployed',
 'read',
 'welfare_income',
 'food_stamp',
 'section_8',
 'times_hospitalized',
 'Lack_concentration',
 'Anxiety',
 'Depression',
 'Obsessive_thinking',
 'Mood_swings',
 'Panic_attacks',
 'Compulsive_behavior',
 'Tiredness',
 'Age',
 'Gender',
 'Household_Income',
 'Region',
 'Device']

In [12]:
del clean_df['unemployed']
del clean_df['Device']
clean_df = clean_df[clean_df.Household_Income != 'Prefer not to answer']
clean_df.head()

,employed,mental_illness,Education,own_computer,hospitalized,hospitalized_days,disabled,internet,with_parents,gap,...,Depression,Obsessive_thinking,Mood_swings,Panic_attacks,Compulsive_behavior,Tiredness,Age,Gender,Household_Income,Region
0,0,0,High School or GED,0,0,0.0,0,1,0,1,...,1,1.0,0.0,1.0,0.0,0.0,30-44,Male,"$25,000-$49,999",Mountain
1,1,1,Some Phd,1,0,0.0,0,1,0,0,...,1,0.0,0.0,1.0,0.0,1.0,18-29,Male,"$50,000-$74,999",East South Central
2,1,0,Completed Undergraduate,1,0,0.0,0,1,0,0,...,0,0.0,0.0,0.0,0.0,0.0,30-44,Male,"$150,000-$174,999",Pacific
3,0,0,Some Undergraduate,1,0,NaN,0,1,1,1,...,0,0.0,0.0,0.0,0.0,0.0,30-44,Male,"$25,000-$49,999",New England
4,1,1,Completed Undergraduate,1,1,35.0,1,1,0,1,...,1,1.0,1.0,1.0,1.0,1.0,30-44,Male,"$25,000-$49,999",East North Central


In [17]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

pd.get_dummies(clean_df, columns=['Education', 'Age', 'Gender', 'Region'])

# labelencoder = LabelEncoder()
# labelencoder.fit(clean_df)

Using TensorFlow backend.


,employed,mental_illness,own_computer,hospitalized,hospitalized_days,disabled,internet,with_parents,gap,months_gap,...,Gender_Male,Region_East North Central,Region_East South Central,Region_Middle Atlantic,Region_Mountain,Region_New England,Region_Pacific,Region_South Atlantic,Region_West North Central,Region_West South Central
0,0,0,0,0,0.0,0,1,0,1,24,...,1,0,0,0,1,0,0,0,0,0
1,1,1,1,0,0.0,0,1,0,0,1,...,1,0,1,0,0,0,0,0,0,0
2,1,0,1,0,0.0,0,1,0,0,0,...,1,0,0,0,0,0,1,0,0,0
3,0,0,1,0,NaN,0,1,1,1,11,...,1,0,0,0,0,1,0,0,0,0
4,1,1,1,1,35.0,1,1,0,1,33,...,1,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328,0,1,0,0,0.0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
330,1,0,1,0,0.0,0,1,1,0,0,...,1,0,0,0,0,0,1,0,0,0
331,1,0,1,0,0.0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
332,0,1,0,1,1.0,1,1,1,1,30,...,0,0,0,0,0,0,0,0,0,1


In [16]:
MultiColumnLabelEncoder(columns = ['Education', 'Age', 'Gender', 'Region']).fit_transform(clean_df)

TypeError: argument must be a string or number

In [5]:
X = clean_df[["employed", "mental_illness", "Education", "own_computer", "hospitalized", "hospitalized_days",
             "disabled", "internet", "with_parents", "gap", "income_with_social", "read",
             "welfare_income", "food_stamp", "section_8", "times_hospitalized", "Lack_concentration", "Depression",
             "Obsessive_thinking", "Mood_swings", "Panic_attacks", "Compulsive_behavior", "Tiredness", "Age", "Gender",
             "Region"]]
y = clean_df["Household_Income"].values.reshape(-1, 1)
print(X.shape, y.shape)

(296, 26) (296, 1)


In [6]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, clean_df['Household_Income'], random_state = 18)

In [7]:
# Create linear regression model

from sklearn.linear_model import LinearRegression
model = LinearRegression()

In [8]:
# Fit the model to the training data and calculate the scores for the training and testing data

model.fit(x_train, y_train)
model.coef_

training_score = model.score(x_train, y_train)
testing_score = model.score(x_test, y_test)

print(f"Training Score: {training_score}")
print(f"Testing Score: {testing_score}")

ValueError: could not convert string to float: 'Completed Masters'